# Código do TCC de Paulo de Tarso, da EMAp FGV

In [37]:
import nltk
#Se for preciso, executar o comando abaixo
#nltk.download()
import re
import os
from sklearn.feature_extraction.text import CountVectorizer as CV

## Códigos auxiliares
### Código para remover símbolos das strings

In [4]:
def clean_string(s):
    #Remover símbolos
    s = re.sub(r'[^\w]', ' ', s)
    #Trocar espaço múltiplo por simples
    s = re.sub( '\s+', ' ', s)
    return s

### Parâmetros

In [17]:
#Parâmetros
inicio_atas = 21
final_atas = 206

total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

### Tokenização dos textos

#### Obter textos dos arquivos

In [39]:
texts = []

#Ir para a pasta com os arquivos
os.chdir(path)

#Obter o texto dos arquivos
for filename in os.listdir():
    file = open(filename)
    texts.append(file.read())
    file.close()
    
#Voltar uma pasta
os.chdir("..")

#### Vetorizar os textos obtidos (TODO: verificar utilizade de todos os argumentos do CountVectorizer e remover as stopwords)

In [40]:
vectorizer = CV()
arrays = vectorizer.fit_transform(texts).toarray()
d_arrays = dict(zip(range(inicio_atas,final_atas+1),arrays))

In [41]:
vectorizer.get_feature_names()

['00',
 '000',
 '00h',
 '01',
 '0104',
 '011',
 '012',
 '0145',
 '0163',
 '0179',
 '018',
 '0180',
 '0183',
 '019',
 '02',
 '026',
 '027',
 '028',
 '03',
 '034',
 '036',
 '04',
 '042',
 '045',
 '046',
 '049',
 '05',
 '051',
 '053',
 '05h',
 '06',
 '064',
 '065',
 '066',
 '067',
 '068',
 '06p',
 '07',
 '070',
 '078',
 '079',
 '08',
 '084',
 '086',
 '088',
 '09',
 '092',
 '093',
 '095',
 '096',
 '10',
 '100',
 '1001',
 '1005',
 '101',
 '102',
 '103',
 '104',
 '1045',
 '105',
 '106',
 '107',
 '108',
 '109',
 '10h',
 '10h02',
 '10h04',
 '10h09',
 '10h13',
 '10h30',
 '10h37',
 '10h48',
 '10º',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '11de',
 '11h',
 '11h15',
 '11h28',
 '11h30',
 '11h39',
 '11h47',
 '11h49',
 '11h57',
 '12',
 '120',
 '120p',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '1280',
 '129',
 '12h',
 '12h03',
 '12h09',
 '12h10',
 '12h23',
 '12h30',
 '12m',
 '12ª',
 '12º',
 '13',
 '130',
 '1300',
 '131',
 '132',
 '1